In [1]:
import requests as req
from bs4 import BeautifulSoup as bs
from tqdm import tqdm_notebook as tqdm
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
import time
import pandas as pd

HTML 태그와 클래스 구분에는 . 을 사용 

그런데 클래스 이름에 . 이 들어가 있으면 어떻게 할까?

\.을 넣으면 문자열 . 이 출력이 되고 띄어쓰기 같은 경우는 .을 쳐보자.

# 페이지 번호의 규칙 찾기

크롤링을 할 때 페이지를 넘겨야 할 때 
각 페이지 수마다 url이 달라지지 않는다면?

Xpath나 css selector의 규칙성을 살펴보고 포맷팅 해야한다.

In [2]:
driver=webdriver.Chrome(ChromeDriverManager().install())
driver.get("https://map.kakao.com/")

In [3]:
search=driver.find_element_by_id("search.keyword.query")

In [4]:
search.send_keys("맛집")

In [5]:
coach=driver.find_element_by_css_selector("body > div.coach_layer.coach_layer_type1 > div > div > div > span")
coach.click()
#버튼 없애기

In [6]:
search.send_keys(Keys.ENTER)

In [7]:
body=driver.find_element_by_css_selector("body")
#스크롤링 실행
time.sleep(1)
body.send_keys(Keys.END)
time.sleep(1)
#아래 더보기버튼 누르기 위해 스크롤링 실행

In [8]:
place=driver.find_element_by_css_selector("#info\.search\.place\.more")
act=ActionChains(driver)
act.click(place)
act.perform()

In [9]:
body=driver.find_element_by_css_selector("body")
#스크롤링 실행
time.sleep(1)
body.send_keys(Keys.END)
time.sleep(1)
#아래 페이지 버튼 눌러야해서 스크롤링 1번 더함.

In [10]:
soup=bs(driver.page_source,"lxml")
#동적 크롤링에서 가져온 페이지를 Beautifulsoup을 사용해 객체화 할 때 이 코드를 입력한다. 

In [11]:
name=soup.select("a.link_name")
name
#전처리

[<a class="link_name" data-id="name" href="#none" title="크래버대게나라 서초점"><span class="ico_ad">광고</span>크래버대게나라 서초점</a>,
 <a class="link_name" data-id="name" href="#none" title="밴건디스테이크하우스">밴건디스테이크하우스</a>,
 <a class="link_name" data-id="name" href="#none" title="브루클린더버거조인트 서래본점">브루클린더버거조인트 서래본점</a>,
 <a class="link_name" data-id="name" href="#none" title="마얘">마얘</a>,
 <a class="link_name" data-id="name" href="#none" title="서래향">서래향</a>,
 <a class="link_name" data-id="name" href="#none" title="비스트로누">비스트로누</a>,
 <a class="link_name" data-id="name" href="#none" title="앙떼띠 팝업스토어">앙떼띠 팝업스토어</a>,
 <a class="link_name" data-id="name" href="#none" title="우참판 서래본점">우참판 서래본점</a>,
 <a class="link_name" data-id="name" href="#none" title="볼라레">볼라레</a>,
 <a class="link_name" data-id="name" href="#none" title="스시타노">스시타노</a>,
 <a class="link_name" data-id="name" href="#none" title="37.5 서래마을점">37.5 서래마을점</a>,
 <a class="link_name" data-id="name" href="#none" title="줄라이">줄라이</a>,
 <a class="link_name" da

In [12]:
name[0].text

'광고크래버대게나라 서초점'

In [13]:
score=soup.select("span.score > em.num")
#전처리

In [14]:
len(score)

16

페이지 번호 클릭

#1
//*[@id="info.search.page.no1"]
#2
//*[@id="info.search.page.no2"]
#3
//*[@id="info.search.page.no3"]

#다음 버튼
//*[@id="info.search.page.next"]

#6
//*[@id="info.search.page.no1"]

#7
//*[@id="info.search.page.no2"]

In [15]:
name_list=[]
score_list=[]
while True:
    try:
        for i in tqdm(range(1,6)):
            page=driver.find_element_by_xpath('//*[@id="info.search.page.no{}"]'.format(i))
            #xpath는 안에 큰따옴표가 있어서 작은 따옴표로 묶어주어야 함
            page.click()

            time.sleep(1)
            name=soup.select("a.link_name")
            for i in name:
                name_list.append(i.text)
            score=soup.select("span.score > em.num")
            for i in score:
                score_list.append(i.text)
            
#             for j in range(len(name)):
#                 name_list.append(name[j].text)
#                 score_list.append(score[j].text)  
#                 전처리를 했을때 name의 길이와 score의 길이가 같았기 때문에 for문을 1번만 써도 됨(권장)
                

        #다음버튼(>)표시
        button=driver.find_element_by_xpath('//*[@id="info.search.page.next"]')
        button.click()
        time.sleep(1)
        name=soup.select("a.link_name")
        for i in name:
            name_list.append(i.text)
        score=soup.select("span.score > em.num")
        for i in score:
            score_list.append(i.text)
    except:
        break

C:\Users\user\AppData\Local\Temp\ipykernel_16972\3881195324.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  while tqdm(True):


0it [00:00, ?it/s]

C:\Users\user\AppData\Local\Temp\ipykernel_16972\3881195324.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i in tqdm(range(1,6)):


  0%|          | 0/5 [00:00<?, ?it/s]

C:\Users\user\AppData\Local\Temp\ipykernel_16972\3881195324.py:12: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i in tqdm(name):


...

  0%|          | 0/16 [00:00<?, ?it/s]

  0%|          | 0/16 [00:00<?, ?it/s]

  0%|          | 0/16 [00:00<?, ?it/s]

  0%|          | 0/16 [00:00<?, ?it/s]

  0%|          | 0/16 [00:00<?, ?it/s]

In [19]:
len(name_list)

640

In [20]:
len(score_list)

640

In [21]:
data={"이름":name_list,"평점":score_list}
kakaomap_df=pd.DataFrame(data)
kakaomap_df

,이름,평점
0,광고크래버대게나라 서초점,3.9
1,밴건디스테이크하우스,4.6
2,브루클린더버거조인트 서래본점,3.9
3,마얘,3.4
4,서래향,2.4
...,...,...
635,줄라이,4.2
636,스와니예,4.2
637,카페노티드 서래,3.2
638,로얄맨션,2.5
